In [ ]:
# Asset Maintenance Model Example: Ensuring java & libraries are available

from javaconnectormulti import AssetMaintenanceModelMultiRunConnector

ammc = AssetMaintenanceModelMultiRunConnector()

print ( ammc.java_details() )

## !!!!! restart kernal between model runs


In [ ]:
# Asset Maintenance Model Example: loading a conf file and run the hypothesis

from javaconnectormulti import AssetMaintenanceModelMultiRunConnector

ammc = AssetMaintenanceModelMultiRunConnector()

ammc.load_config( "conf/simpleexperiment1.xlsx" )

ammc.run_once()

In [ ]:
# Once off Asset Maintenance Model Example: Setting conf variables manually
# better to use Multi-run like below

from javaconnectormulti import AssetMaintenanceModelMultiRunConnector
from jpype import *

ammc = AssetMaintenanceModelMultiRunConnector()

ammc.load_config( "conf/simpleexperiment1.xlsx" )

# Change a separation rate

# first get pool

config = ammc.origin_conf;
        
tlevel = config.getRootConfig().workforce.get_from_all_technicians_levels_OrNull( "APS_Maintainer" );

AssetsMaintenanceConfig = JClass( "edu.unsw.csc.rbm.gm.utility.config.AssetsMaintenanceConfig" )

# then set separation rate, have to divide down from yearly rate to hourly rate and use constant table function
sp_rate = 0.01 / ( 365.0 * 24.0)
tlevel.separation_rate = AssetsMaintenanceConfig.makeConstantTableFunction( sp_rate )

# then set recruitment rate, have to divide down from yearly rate to hourly rate and use constant table function
rec_rate = 1.5 / ( 365.0 * 24.0)
tlevel.recruitment_rate = AssetsMaintenanceConfig.makeConstantTableFunction( rec_rate )

tlevel.promotion_min_tasks = 20

# have to convert to hours from days
promotiondays = 30
tlevel.promotion_min_tenure = promotiondays * 24

# add to schedule
# create a object for the schedule
asset_id = "TESTASSET"
start_usage_str = "EXERCISE"
end_usage_str = "EXERCISE"
end_facility_str = "STORAGE1"
maint_facility_str = "FACILITY1"
start_facility_str = "STORAGE1"
maintCfg = config.createMaintenanceConfig(asset_id, start_usage_str, end_usage_str, end_facility_str, maint_facility_str, start_facility_str)

# add repeating schedule
# six times a year requests two maintenances occur
numEventsPerYear = 6
numAssetsPerEvent = 2
config.addAssetConfigs( maintCfg, "20/01/2020", numEventsPerYear, numAssetsPerEvent )

# add one off schedule
config.addAssetConfig( maintCfg, "20/01/2020", numAssetsPerEvent )

# get an asset class by name
acc = config.getRootConfig().section_asset_module.get_from_asset_classes_OrNull("TESTASSET")
# get a task description by name
td = acc.maintenance_strategy.get_from_all_tasks_OrNull("UNPACK")
# set the Task competency repeats required.
td.instances_required = 5

main = ammc.run_once()

#  Output

# Total cost
dataset1 = main.rootFactor.task_module.ds_cost
dataset2 = main.rootFactor.workforce.ds_monthly_cost

ac = main.rootFactor.section_asset_module.get_from_asset_classes("TESTASSET")

# Scheduled maintenance compliance
dataset3 = ac.ds_on_time_maintenance
dataset4 = ac.ds_scheduled_maintenance
# Scheduled maintenance completed
dataset5 = ac.ds_completed_on_time_maintenance
dataset6 = ac.ds_completed_scheduled_maintenance

# tasks backlog
dataset7 = main.rootFactor.get_task_module().ds_tasks_in_backlog

# Active technicians
dataset8 = main.rootFactor.get_workforce().ds_active_technicians


# Promoted technicians
dataset9 = main.rootFactor.get_workforce().ds_monthlyPromoted

helper = ammc.Helper(main)

# All Techs
datasetalltechs = helper.getAllTechnitions()
# Idle Techs
datasetidle = helper.getIdle()

print("technitions")
print(datasetalltechs.getData().size())
print(datasetidle.getData().size())

for idx in range (0, datasetalltechs.getData().size()):
    print ( datasetalltechs.getData().get(idx).xval,  datasetalltechs.getData().get(idx).yval, datasetidle.getData().get(idx).yval )

In [ ]:
# Multi Run Asset Maintenance Model Example

from javaconnectormulti import AssetMaintenanceModelMultiRunConnector

ammc = AssetMaintenanceModelMultiRunConnector()

ammc.load_config( "conf/simpleexperiment1.xlsx" )

# set up two replications of the config
rep1 = ammc.add_replication()
rep2 = ammc.add_replication()

# change the config before running
rep2.getConf().getRootConfig().workforce.maxTopLevelCompetency = 3

# run both at once one per thread
ammc.run_multi()

# get the results
helper1 = ammc.Helper( rep1) 
manager_pool1 = helper1.getPool("APS_Manager")
assert manager_pool1.technicians.size() == 1

helper2 = ammc.Helper( rep2 )
manager_pool2 = helper2.getPool("APS_Manager")
assert manager_pool2.technicians.size() == 3

print ("done")

In [1]:
# Asset Maintenance Model Example: Setting conf variables manually

import numpy as np
from javaconnectormulti import AssetMaintenanceModelMultiRunConnector
from jpype import *
import numpy as np


ammc = AssetMaintenanceModelMultiRunConnector()

ammc.load_config( "conf/trial.xlsx" )

# make local variables
helper = ammc.origin_conf
config = helper.getRootConfig()

# Change a separation rate

# first get pool
        
tlevel = config.workforce.get_from_all_technicians_levels_OrNull( "APS_Maintainer" );
tlevel2 = config.workforce.get_from_all_technicians_levels_OrNull( "APS_Verifier" );
tlevel2.promotion_min_tasks = 20

AssetsMaintenanceConfig = JClass( "edu.unsw.csc.rbm.gm.utility.config.AssetsMaintenanceConfig" )

# then set separation rate, have to divide down from yearly rate to hourly rate and use constant table function
sp_rate = 0.01 / ( 365.0 * 24.0)
tlevel.separation_rate = AssetsMaintenanceConfig.makeConstantTableFunction( sp_rate )

# then set recruitment rate, have to divide down from yearly rate to hourly rate and use constant table function
rec_rate = 1.5 / ( 365.0 * 24.0)
tlevel.recruitment_rate = AssetsMaintenanceConfig.makeConstantTableFunction( rec_rate )

tlevel.promotion_min_tasks = 20

# have to convert to hours from days
promotiondays = 30
tlevel.promotion_min_tenure = promotiondays * 24

# add to schedule
# create a object for the schedule
asset_id = "TESTASSET"
start_usage_str = "EXERCISE"
end_usage_str = "EXERCISE"
end_facility_str = "STORAGE1"
maint_facility_str = "FACILITY1"
start_facility_str = "STORAGE1"

maintCfg = helper.createMaintenanceConfig(asset_id, start_usage_str, end_usage_str, end_facility_str, maint_facility_str, start_facility_str)


# add repeating schedule
# six times a year requests two maintenances occur
numEventsPerYear = 6
numAssetsPerEvent = 2
helper.addAssetConfigs( maintCfg, "20/01/2020", numEventsPerYear, numAssetsPerEvent )

# add one off schedule
helper.addAssetConfig( maintCfg, "20/01/2020", numAssetsPerEvent )

# get an asset class by name
acc = config.section_asset_module.get_from_asset_classes_OrNull("TESTASSET")
# get a task description by name
td = acc.maintenance_strategy.get_from_all_tasks_OrNull("UNPACK")

# set the Task competency repeats required.
td.instances_required = 5

# Run once , get back results
main = ammc.run_once()

#  Output

# Total cost
dataset1 = main.rootFactor.task_module.ds_cost
dataset2 = main.rootFactor.workforce.ds_monthly_cost

ac = main.rootFactor.section_asset_module.get_from_asset_classes("TESTASSET")

# Scheduled maintenance compliance
dataset3 = ac.ds_on_time_maintenance
dataset4 = ac.ds_scheduled_maintenance
# Scheduled maintenance completed
dataset5 = ac.ds_completed_on_time_maintenance
dataset6 = ac.ds_completed_scheduled_maintenance

# tasks backlog
dataset7 = main.rootFactor.get_task_module().ds_tasks_in_backlog

# Active technicians
dataset8 = main.rootFactor.get_workforce().ds_active_technicians

# Promoted technicians
dataset9 = main.rootFactor.get_workforce().ds_monthlyPromoted


x=[];  #list type
print(dataset1.getData().size())

for ind in range(0,dataset1.getData().size()):
    x.append(dataset1.getData().get(ind).yval)

cost=np.array(x)
print(cost.sum())


522
78090.0


In [ ]:
#a=[]
#for i in range(dataset4.getData().size()):
#    a.append(dataset4.getData().get(i).yval)

In [ ]:
#import matplotlib.pyplot as plt
#plt.plot(a)

In [ ]:
#import numpy as np

#np.mean(a)